In [2]:
import os
import re
import json
import torch
from random import seed, shuffle
from collections import Counter
from openai import OpenAI
from time import time
import pandas as pd
import json
from datasets import load_dataset
from pprint import pprint
from time import time
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import sys
from time import time
from tqdm import tqdm
import random 
import json
sys.path.insert(0, '../KGWatermark')
from extended_watermark_processor import WatermarkLogitsProcessor
from extended_watermark_processor import WatermarkDetector
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LogitsProcessorList
from transformers import GenerationConfig
from datasets import load_dataset
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
import os
import re
import copy
import json
from extended_watermark_processor import WatermarkDetector
import time
import requests
from tqdm import tqdm
from random import shuffle
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from requests.exceptions import ConnectionError
import numpy as np
from GET_API_KEY import GET_OPENAI_API_KEY
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/ShenChenchen/.conda/envs/UnbiasedWatermark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [4]:
client = OpenAI(api_key=GET_OPENAI_API_KEY())
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
# Test the model's performance and connection status
print(completion.choices[0].message)

APIConnectionError: Connection error.

In [ ]:
gpt4_name = 'gpt-4-0613'
gpt35_name = 'gpt-3.5-turbo-1106'
temperature = 0.0
openai_api_key = GET_OPENAI_API_KEY()  # use your api key
gpt4 = ChatOpenAI(model_name=gpt4_name, temperature=temperature,
                    openai_api_key=openai_api_key, request_timeout=120)
gpt35 = ChatOpenAI(model_name=gpt35_name, temperature=temperature,
                    openai_api_key=openai_api_key, request_timeout=120)

In [5]:
prompt_string  = "I want you to add in a prefix to the following text, you don't need to care about grammer or punctuation, you need to output the exact same content after the given prefix.\
[Prefix]:{prefix}, [Content]: {content} [Repetition]:"
chat_prompt = ChatPromptTemplate.from_template(prompt_string)
prefix_list = [
    'I am telling you a story:',
    'Hi, let me share something with you:',
    'Remember this:',
    'Just imagine:',
    'Once upon a time,',
    'Let me explain:',
    'Listen carefully:',
    "Here's a little secret:",
    'Think about this:',
    'Do you know about'
]
# _output = gpt4(chat_prompt.format_messages(prefix=prefix_lists[0], content= 'how are you?'))

In [6]:
dataset = load_dataset("allenai/c4", "realnewslike")

Using the latest cached version of the dataset since allenai/c4 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'realnewslike' at /home/ShenChenchen/.cache/huggingface/datasets/allenai___c4/realnewslike/0.0.0/1588ec454efa1a09f29cd18ddd04fe05fc8653a2 (last modified on Tue Mar 26 12:00:09 2024).


In [7]:
# output_list = [
#     gpt4(chat_prompt.format_messages(prefix=prefix_lists[0], content=repitition_content[0])).content
# ]

def compare_content_with_prefix(prefix, expected_content, model_output):
    # Check if the model output starts with the prefix
    if not model_output.startswith(prefix):
        return False, "Output does not start with the given prefix."
    
    # Remove the prefix from the model output to get the content part
    content_start_index = len(prefix)
    actual_content = model_output[content_start_index:]

    expected_content = expected_content.strip().lower().replace('\n', ' ')
    actual_content = actual_content.strip().lower().replace('\n', ' ')
    
    # Now compare the actual content with the expected content
    if actual_content == expected_content:
        return True, "Content matches exactly."
    else:
        # print(actual_content.strip().lower().replace('\n', ' '))
        # print(expected_content.strip().lower())
        
        # Find the first mismatch if content does not match exactly
        min_length = min(len(expected_content), len(actual_content))
        for i in range(min_length):
            if expected_content[i] != actual_content[i]:
                print(f"Expected:{expected_content[i]}, Actual:{actual_content[i]},{i}")
                mismatch_index = i
                break
        else:
            # If no mismatch found in common length, check for length mismatch
            if len(expected_content) != len(actual_content):
                mismatch_index = min_length
            else:
                # Identical content
                return True, "Content matches exactly."
        
        # Highlight the mismatch in expected and actual contents
        expected_highlight = expected_content[:mismatch_index] + "[" + expected_content[mismatch_index] + "]" + expected_content[mismatch_index+1:]
        actual_highlight = actual_content[:mismatch_index] + "[" + actual_content[mismatch_index] + "]" + actual_content[mismatch_index+1:]
        
        mismatch_info = f"Mismatch found at character {mismatch_index + 1} of the content part.\nExpected: {expected_highlight}\nActual: {actual_highlight}"
        return False, mismatch_info, expected_content, actual_content


In [8]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b", device_map = 'auto')
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b",quantization_config = bnb_config, device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/ShenChenchen/.conda/envs/UnbiasedWatermark/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


### GPT Repetition

In [ ]:
# Example usage
# prefix = "This is a prefix: "
# expected_content = "This is the expected content."
# model_output = "This is a prefix: This is the expexted content."

# model_output = [gpt4(chat_prompt.format_messages(prefix=prefix, content=content)).content for content in repitition_content for prefix in prefix_lists[:2]]
batch_size = 10
epoch = 20
test = 0
test_limit = 2
repetition_content = dataset['validation']['text'][:epoch]

_input_list = []
for prefix in prefix_list:
    for  content in repetition_content:
        _input_list.append(chat_prompt.format_messages(prefix=prefix, content=content))
print(len(_input_list))

_output_list = []
for i in tqdm(range(0, len(_input_list),batch_size), desc="Processing"):
    # _input = [[msg.content for msg in input_batch] for input_batch in _input_list[i:i+batch_size]]
    # print(_input)
    _output_list.extend([o1.content for o1 in gpt4.batch(_input_list[:batch_size])])
    # break
    # print(_outputs)
    # test += 1
    # if test == test_limit:
    #     break
    
    

# for i in range(len(model_output)):
#     match, message = compare_content_with_prefix(prefix_lists[i], repitition_content[i], model_output[i])
#     print(f"Match: {match}\nMessage: {message}")


In [17]:
epoch = 5
repetition_content = dataset['validation']['text'][:epoch]
_input_list= []
for prefix in prefix_list:
    for  content in repetition_content:
        _input_list.append(prompt_string.format(prefix=prefix, content=content))
pprint(_input_list)

["I want you to add in a prefix to the following text, you don't need to care "
 'about grammer or punctuation, you need to output the exact same content '
 'after the given prefix.[Prefix]:I am telling you a story:, [Content]: '
 "Cluster comprises IBM's Opteron-based eServer 325 server and systems "
 'management software and storage devices that can run Linux and Windows '
 'operating systems.\n'
 'IBM on Tuesday announced a prepackaged and pretested cluster that is powered '
 'by Advanced Micro Devices Inc.s 64-bit Opteron processor.\n'
 'IBM, of Armonk, N.Y., is combining its Opteron-based eServer 325 '
 'server—which targets scientific and technical computing users—with systems '
 'management software and storage devices to offer a bundled package that can '
 'run Linux and Windows operating systems.\n'
 'The new offering, which will be available later this month or in early '
 'December, is part of IBMs eServer Cluster 1350, which also includes bundles '
 'based on the companys B

### OPT Repetition

In [ ]:
epoch = 20
test = 0
test_limit = 2
repetition_content = dataset['validation']['text'][:epoch]

print("Start Tokenization...")
tokenized_tensors = list(map(lambda s: tokenizer(s, return_tensors='pt'), dataset['validation']['text']))
print("End Tokenization...")

In [ ]:
for i in range(len(_output_list)):
    print(compare_content_with_prefix(prefix_list[0], repetition_content[i],_output_list[i]))
# C[700:756]

In [ ]:
len(_output_list)

2

In [ ]:
pprint(compare_content_with_prefix(prefix_list[0], repetition_content[1],_outputs[1]))

belying expectations, prasar bharti has earned only rs 58.19 crore (rs 581.9 million) as revenue during the commonwealth games last month. the gross revenue earned by pb on account of telecasting/broadcasting of advertisements on doordarshan channel and all india radio during coverage of the commonwealth games is rs 58.17 crore, minister of state for information and broadcasting s jagathrakshakan informed the lok sabha on tuesday. while air earned rs 2.18 crore (rs 21.8 million), doordarshan garnered rs 55.99 crore (rs 559.9 million) as revenue, he said. prasar bharati had earlier said it knew in advance that the recently concluded commonwealth games, for which doordarshan was the official broadcaster, would not bring in huge advertising revenues as a result of luke warm response from advertisers. notably, dd was not allowed to air advertisements during the closing ceremony of the cwg games following complaints that the opening ceremony had been shown deferred live to accommodate adver

In [ ]:
_outputs[1]

'I am telling you a story: Belying expectations, Prasar Bharti has earned only Rs 58.19 crore (Rs 581.9 million) as revenue during the Commonwealth Games last month. The gross revenue earned by PB on account of telecasting/broadcasting of advertisements on Doordarshan channel and All India Radio during coverage of the Commonwealth Games is Rs 58.17 crore, Minister of State for Information and Broadcasting S Jagathrakshakan informed the Lok Sabha on Tuesday. While AIR earned Rs 2.18 crore (Rs 21.8 million), Doordarshan garnered Rs 55.99 crore (Rs 559.9 million) as revenue, he said. Prasar Bharati had earlier said it knew in advance that the recently concluded Commonwealth Games, for which Doordarshan was the official broadcaster, would not bring in huge advertising revenues as a result of luke warm response from advertisers. Notably, DD was not allowed to air advertisements during the closing ceremony of the CWG Games following complaints that the opening ceremony had been shown deferre

In [ ]:
print(type(result), len(result))
pprint(result)